# Introdução

Este notebook tem como objetivo desenvolver e avaliar modelos de classificação
para prever a gravidade dos acidentes de trânsito, utilizando dados tratados
da Polícia Rodoviária Federal (PRF) e Ministerio do transporte.

O foco do estudo é identificar acidentes classificados como graves
(feridos graves ou óbitos), a partir de variáveis relacionadas ao contexto,
local e características do acidente.


In [28]:
import pandas as pd
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

# Carregamento e inspeção dos dados

Os dados utilizados neste estudo foram previamente tratados e armazenados em formato CSV.
Nesta etapa, o dataset é carregado e analisado quanto à sua estrutura,
tipos de dados e quantidade de registros.


In [4]:
df = pd.read_csv('../data/tratados/amostra_19-21.csv', sep= ',', encoding="ISO-8859-1")

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114536 entries, 0 to 114535
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   uf                      114536 non-null  object 
 1   br                      114297 non-null  float64
 2   km                      114297 non-null  object 
 3   municipio               114536 non-null  object 
 4   dia_semana              114536 non-null  object 
 5   fase_dia                114536 non-null  object 
 6   sentido_via             114536 non-null  object 
 7   condicao_metereologica  114536 non-null  object 
 8   tipo_pista              114536 non-null  object 
 9   tracado_via             114536 non-null  object 
 10  uso_solo                114536 non-null  object 
 11  tipo_veiculo            110217 non-null  object 
 12  data_inversa            114536 non-null  object 
 13  ano                     114536 non-null  int64  
 14  frota               

# Modelo 1 – Regressão Logística

A Regressão Logística foi utilizada como modelo base por ser amplamente aplicada
em problemas de classificação binária e por permitir boa interpretabilidade.

O uso do parâmetro `class_weight='balanced'` foi necessário devido ao
desbalanceamento das classes.


In [ ]:

# 2. Separação das variáveis (exemplo)

X = df.drop('gravidade', axis=1)
y = df['gravidade']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 3. Definir colunas numéricas e categóricas

numeric_features = ['br', 'frota']
categorical_features = [
    'uf', 'km', 'municipio', 'dia_semana', 'fase_dia',
    'sentido_via', 'condicao_metereologica', 'tipo_pista',
    'tracado_via', 'uso_solo', 'tipo_veiculo']

# 4. Pré-processamento

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocess = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# 5. Pipeline final

pipeline = Pipeline(steps=[
    ('preprocess', preprocess),
    ('clf', LogisticRegression(max_iter=300))
])

# 6. Grade de hiperparâmetros

param_grid = {
    'clf__C': [0.01, 0.1, 1, 10],
    'clf__solver': ['liblinear', 'saga'],
    'clf__penalty': ['l1', 'l2'],
    'clf__class_weight': [None, 'balanced']
}

# 7. GridSearchCV

grid = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='f1',      
    cv=3,
    verbose=1,
    n_jobs=-1
)

# Treino
grid.fit(X_train, y_train)

# 8. Resultados

print("Melhores parâmetros:")
print(grid.best_params_)

y_pred = grid.predict(X_test)

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

print("\nMatriz de Confusão:")
print(confusion_matrix(y_test, y_pred))


Fitting 3 folds for each of 32 candidates, totalling 96 fits
Melhores parâmetros:
{'clf__C': 0.1, 'clf__class_weight': 'balanced', 'clf__penalty': 'l2', 'clf__solver': 'liblinear'}

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.93      0.69      0.79     19305
           1       0.30      0.71      0.42      3603

    accuracy                           0.70     22908
   macro avg       0.61      0.70      0.61     22908
weighted avg       0.83      0.70      0.73     22908


Matriz de Confusão:
[[13368  5937]
 [ 1046  2557]]


Recall da classe grave: ≈ 0,71

F1-score da classe grave: ≈ 0,42

Accuracy: ≈ 0,70

O modelo apresenta boa capacidade de identificar acidentes graves, mas com baixa precisão, indicando um número elevado de falsos positivos. Ainda assim, é um baseline adequado e coerente com o problema.

# Modelo 2 – XGBoost

O XGBoost é um algoritmo baseado em árvores de decisão e boosting,
sendo conhecido por seu bom desempenho em problemas com dados tabulares
e relações não lineares.


In [ ]:

X = df.drop('gravidade', axis=1)
y = df['gravidade']

# Separar por tipo
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()


# 2. Pré-processamento


preprocess = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=True), categorical_cols),
        ('num', 'passthrough', numeric_cols)
    ]
)
# 3. Divisão treino/teste


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


# 4. XGBOOST — com parâmetros básicos


# Cálculo do scale_pos_weight (importante!)
ratio = y_train.value_counts()[0] / y_train.value_counts()[1]
print("scale_pos_weight =", ratio)

xgb = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    tree_method='hist',      
    scale_pos_weight=ratio,   # Ajuste para desbalanceamento
)


# 5. Pipeline


pipeline = Pipeline(steps=[
    ('preprocess', preprocess),
    ('clf', xgb)
])

# 6. Grade de hiperparâmetros


param_grid = {
    'clf__n_estimators': [200, 500],
    'clf__max_depth': [4, 6, 8],
    'clf__learning_rate': [0.01, 0.1],
    'clf__subsample': [0.8, 1.0],
    'clf__colsample_bytree': [0.7, 1.0]
}


# 7. Grid Search

grid = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='f1_macro',
    cv=3,
    verbose=2,
    n_jobs=-1
)

grid.fit(X_train, y_train)


# 8. Avaliação do Modelo


print("\nMelhores parâmetros encontrados:")
print(grid.best_params_)

y_pred = grid.predict(X_test)

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

print("\nMatriz de Confusão:")
print(confusion_matrix(y_test, y_pred))


scale_pos_weight = 5.3586398334489935
Fitting 3 folds for each of 48 candidates, totalling 144 fits


c:\Users\Elaine\Desktop\TCC_acidentes\venv\Lib\site-packages\xgboost\training.py:199: UserWarning: [22:09:54] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



Melhores parâmetros encontrados:
{'clf__colsample_bytree': 0.7, 'clf__learning_rate': 0.1, 'clf__max_depth': 8, 'clf__n_estimators': 500, 'clf__subsample': 0.8}

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.93      0.73      0.82     19305
           1       0.33      0.73      0.46      3603

    accuracy                           0.73     22908
   macro avg       0.63      0.73      0.64     22908
weighted avg       0.84      0.73      0.76     22908


Matriz de Confusão:
[[14026  5279]
 [  982  2621]]


Recall da classe grave: ≈ 0,73

F1-score da classe grave: ≈ 0,46

Accuracy: ≈ 0,73

O XGBoost supera a Regressão Logística em todas as métricas relevantes. O uso de scale_pos_weight mostrou-se fundamental para lidar com o desbalanceamento, resultando em melhor equilíbrio entre identificação de casos graves e controle de erros.

# StratifiedKFold + Regressão Logística

In [ ]:

# 2. Identificar tipos de colunas

num_cols = X.select_dtypes(include=["int64", "float64"]).columns
cat_cols = X.select_dtypes(include=["object"]).columns

print("Numéricas:", len(num_cols))
print("Categóricas:", len(cat_cols))

# 3. Preprocessamento: imputação + encoding + scaling

preprocess = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler())
        ]), num_cols),

        ("cat", Pipeline([
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("onehot", OneHotEncoder(handle_unknown="ignore"))
        ]), cat_cols)
    ]
)

# 4. Train/test split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 5. StratifiedKFold

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)


# 6. PIPELINE + LOGISTIC REGRESSION

pipe_lr = Pipeline([
    ("prep", preprocess),
    ("clf", LogisticRegression(max_iter=500))
])

param_lr = {
    "clf__C": np.logspace(-3, 2, 20),
    "clf__penalty": ["l2"],
    "clf__class_weight": [None, "balanced"],
    "clf__solver": ["liblinear", "lbfgs"]
}

search_lr = RandomizedSearchCV(
    pipe_lr,
    param_distributions=param_lr,
    n_iter=30,
    scoring="f1",
    cv=skf,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

print("\n===== TREINANDO LOGISTIC REGRESSION =====")
search_lr.fit(X_train, y_train)

print("\n=== MELHORES PARÂMETROS (LR) ===")
print(search_lr.best_params_)

best_lr = search_lr.best_estimator_

y_pred_lr = best_lr.predict(X_test)

print("\n=== RELATÓRIO DE CLASSIFICAÇÃO (LR) ===")
print(classification_report(y_test, y_pred_lr))

print("\n=== MATRIZ DE CONFUSÃO (LR) ===")
print(confusion_matrix(y_test, y_pred_lr))



Numéricas: 4
Categóricas: 10

===== TREINANDO LOGISTIC REGRESSION =====
Fitting 3 folds for each of 30 candidates, totalling 90 fits

=== MELHORES PARÂMETROS (LR) ===
{'clf__solver': 'lbfgs', 'clf__penalty': 'l2', 'clf__class_weight': 'balanced', 'clf__C': np.float64(0.12742749857031335)}

=== RELATÓRIO DE CLASSIFICAÇÃO (LR) ===
              precision    recall  f1-score   support

           0       0.93      0.69      0.79     19305
           1       0.30      0.70      0.42      3603

    accuracy                           0.69     22908
   macro avg       0.61      0.69      0.60     22908
weighted avg       0.83      0.69      0.73     22908


=== MATRIZ DE CONFUSÃO (LR) ===
[[13266  6039]
 [ 1072  2531]]


Recall da classe grave: ≈ 0,70

F1-score da classe grave: ≈ 0,42

Accuracy: ≈ 0,69

A validação cruzada estratificada confirma que o desempenho da Regressão Logística é estável, porém limitado. O modelo não apresenta ganhos relevantes em relação à abordagem simples treino–teste.

# StratifiedKFold + XGBoost

In [ ]:
# 7. PIPELINE + XGBOOST

pipe_xgb = Pipeline([
    ("prep", preprocess),
    ("clf", XGBClassifier(
        objective="binary:logistic",
        scale_pos_weight=y_train.value_counts()[0] / y_train.value_counts()[1],
        eval_metric="logloss",
        use_label_encoder=False,
        tree_method="hist"
    ))
])

param_xgb = {
    "clf__n_estimators": [200, 300, 500],
    "clf__max_depth": [4, 6, 8],
    "clf__learning_rate": [0.01, 0.05, 0.1],
    "clf__subsample": [0.6, 0.8, 1.0],
    "clf__colsample_bytree": [0.6, 0.8, 1.0]
}

search_xgb = RandomizedSearchCV(
    pipe_xgb,
    param_distributions=param_xgb,
    n_iter=30,
    scoring="f1",
    cv=skf,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

print("\n===== TREINANDO XGBOOST =====")
search_xgb.fit(X_train, y_train)

print("\n=== MELHORES PARÂMETROS (XGBoost) ===")
print(search_xgb.best_params_)

best_xgb = search_xgb.best_estimator_

y_pred_xgb = best_xgb.predict(X_test)

print("\n=== RELATÓRIO DE CLASSIFICAÇÃO (XGB) ===")
print(classification_report(y_test, y_pred_xgb))

print("\n=== MATRIZ DE CONFUSÃO (XGB) ===")
print(confusion_matrix(y_test, y_pred_xgb))



===== TREINANDO XGBOOST =====
Fitting 3 folds for each of 30 candidates, totalling 90 fits


c:\Users\Elaine\Desktop\TCC_acidentes\venv\Lib\site-packages\xgboost\training.py:199: UserWarning: [00:17:04] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



=== MELHORES PARÂMETROS (XGBoost) ===
{'clf__subsample': 1.0, 'clf__n_estimators': 500, 'clf__max_depth': 8, 'clf__learning_rate': 0.1, 'clf__colsample_bytree': 0.8}

=== RELATÓRIO DE CLASSIFICAÇÃO (XGB) ===
              precision    recall  f1-score   support

           0       0.93      0.73      0.82     19305
           1       0.33      0.72      0.45      3603

    accuracy                           0.73     22908
   macro avg       0.63      0.72      0.63     22908
weighted avg       0.84      0.73      0.76     22908


=== MATRIZ DE CONFUSÃO (XGB) ===
[[14017  5288]
 [ 1006  2597]]


Recall da classe grave: ≈ 0,73

F1-score da classe grave: ≈ 0,46

Accuracy: ≈ 0,73

Essa abordagem apresenta os resultados mais consistentes e robustos. A validação cruzada reduz a dependência de uma única divisão dos dados e confirma a superioridade do XGBoost para o problema analisado.